In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import gc; gc.enable()

from utilities import DfLowMemory

In [2]:
df = DfLowMemory('train_ver2.csv')

In [3]:
# ind_empleado - 27734. Employee index: A active, B ex employed, F filial, N not employee, P pasive. 
#I am noticing that there is a value that is not in the description so I believe it is a typo. 
#I tried looking more into the data but in the end I decided to drop since it is just one person.

df = df[df['ind_empleado'] != 'S']

In [4]:
# pais_residencia - 27734. One aspect that I found interesting is that customers with pais_residenciamissing had all other features missing. 
#For now I am going to drop all of them

df.dropna(subset=['pais_residencia'], inplace=True)

In [5]:
# Now there are 70 missing values for gender. The value distribution doesn't seem very different. I am going to replace them with the mode.
df['sexo'].fillna(df['sexo'].mode()[0], inplace=True)

In [6]:
# ult_fec_cli_1t has a lot of missing values. This represents the last day the the customer was the primary costumer.
# Looking at the indrel_1mes column, it shows that most of the customers at the beginning of the month were the primary costumers. So 
# I am assuming that the customers with missing last date as the primary costumers are still the primary costumers. So I am going to impute 
# this with 'primary'
# THIS NEEDS TO BE MODIFIED TO TYPE DATE
df.loc[df['ult_fec_cli_1t'].isnull(), 'ult_fec_cli_1t'] = 'PRIMARY'

In [7]:
# Customer type at the beginning of the month ,1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner).
# some values that were supposed to be 1 are 1.0 and for the other categories as well. So I am going to use 1,2,3,4 like in the description
# This has been suggested by @StephenSmith
map_dict = {'1.0' : '1',
            '1' : '1',
            '2' : '2',
            '2.0' : '2',
            '3' : '3',
            '3.0' : '3', 
            '4' : '4',
            '4.0' : '4'}

df.indrel_1mes.fillna('P', inplace=True)
df.indrel_1mes = df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
df.indrel_1mes = df.indrel_1mes.astype('category')

In [8]:
# Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential)
# There are some described as N that doesn't fit into the column description.
# Taking a closer look at these 4 rows and comparing them with the ind_actividad: df[df['tiprel_1mes'] == 'N'].iloc[:,8:24]
# The ones that had ind_actividad = 1 were imputed as active and the ones that had 0 were imputed as I
# For the nan values, there is another column named 'ind_actividad_cliente' that is 1 for active costumers and 0 for inactive. I am 
# going to use this to impute the nan values for tiprel_1mes

df.loc[df['tiprel_1mes'].isnull(), 'tiprel_1mes'] = df['ind_actividad_cliente']

map_tip = {1 : 'A',
           0 : 'I'}        

df.tiprel_1mes = df.tiprel_1mes.apply(lambda x: map_tip.get(x,x))
df.loc[6603017, 'tiprel_1mes'] = 'A'
df.loc[10123924, 'tiprel_1mes'] = 'A'
df.loc[10124648, 'tiprel_1mes'] = 'I'
df.loc[11247349, 'tiprel_1mes'] = 'I'

df.tiprel_1mes = df.tiprel_1mes.astype('category')

In [9]:
# conyuemp Spouse index. 1 if the customer is spouse of an employee
# I am assuming that most customers are not spouses and I am going to impute these with the mode
# Now there are 70 missing values for gender. The value distribution doesn't seem very different. I am going to replace them with the mode.
df['conyuemp'].fillna(df['conyuemp'].mode()[0], inplace=True)
df.conyuemp = df.conyuemp.astype('category')

In [10]:
# canal_entrada. Channel used by the customer to join
# I am going to fill the missing values with the mode
df['canal_entrada'].fillna(df['canal_entrada'].mode()[0], inplace=True)

In [11]:
# tipodom. Addres type. 1, primary address. It is only 1 value missing so I am just going to replace it with the mode
df['tipodom'].fillna(df['tipodom'].mode()[0], inplace=True)

In [12]:
# cod_prov. Province code (customer's address)
# nomprov. Province name. 
# These two have the same number of missing values. I am going to replace these with unknow
# THESE IS SUGGESTED BY ALAN PRYOR

for c in ['cod_prov', 'nomprov']:
    df.loc[df[c].isnull(), c ] = 'UNKOWN'

In [13]:
# Renta. Gross income of the household. I am going to replace the missing values with the mean salary per providence. 

salaries = dict(df.groupby('nomprov')['renta'].mean().round(0))
df.loc[df['renta'].isnull(), 'renta'] = df['nomprov']
df.renta = df.renta.apply(lambda x: salaries.get(x,x))

In [15]:
# Segmento. segmentation: 01 - VIP, 02 - Individuals 03 - college graduated
for c in ['segmento']:
    df.loc[df[c].isnull(), c ] = 'UNKOWN'

In [21]:
df['ind_nomina_ult1'].fillna(df['ind_nomina_ult1'].mode()[0], inplace=True)

In [22]:
df['ind_nom_pens_ult1'].fillna(df['ind_nom_pens_ult1'].mode()[0], inplace=True)

In [24]:
df['fecha_dato'] = pd.to_datetime(df['fecha_dato'], format="%Y-%m-%d")
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format="%Y-%m-%d")

In [31]:
df.loc[df['antiguedad'] == '-999999', 'antiguedad'] = 'UNKNOWN'

# Missing values and action plan

ind_empleado - 27734. Employee index: A active, B ex employed, F filial, N not employee, P pasive. I am noticing that there is a value that is not in the description so I believe it is a typo. I tried looking more into the data but in the end I decided to drop since it is just one person. The nan values I am replacing with the mode which in this case it is that the person is not an employee.

df = df[df['ind_empleado'] != 'S']

-------------------------------------------------------
pais_residencia - 27734. One aspect that I found interesting is that customers with pais_residenciamissing had all other features missing. For now I am going to drop all of them

filtered_df = df[df['pais_residencia'].isnull()]
len(filtered_df['ncodpers'].unique())
This shows that there are 7340 without much information about them

-------------------------------------------------------
sexo - 27804. Most of the clients that have this information missing dont have much information on them overall. If we were to drop the ones that have most information missing, sexo would have about 70 values missing. For now I am going to change these with the mode.

-------------------------------------------------------
I need to turn fecha_dato and fecha_alta into dates

fecha_alta - 27734. This is the date that the customer joined the bank.
Added another column called month. May be useful in the case that some customers buy more depending on the time of the year.

-------------------------------------------------------
ult_fec_cli_1t - 13622516. This is the last date as a primary customer.
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_nomina_ult1             16063
ind_nom_pens_ult1           16063


-------------------------------------------------------
A bank guarantee is a lending institution's promise to cover a loss if a borrower (their customer) defaults on a debt to a third party. 

A payroll account is a separate bank account for your business that is strictly used for payroll. Instead of lumping all your business expenses into one account, you will pay employee wages with your payroll bank account. The money going into the payroll account will only be used for payroll.

Bank policy requires that account owners be the legal age of 18 in order to open an account online. If you are not 18 years old, you may open an account for minors with your parent or legal guardian as the joint account holder.

A pension plan is a type of retirement plan where an employee adds money into a fund that includes contributions by the employer.

A securities account sometimes known as a brokerage account is an account that holds financial assets such as securities on behalf of an investor with a bank, broker or custodian.

In [35]:
used_cols = [c for c in df.columns.tolist() if c not in ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']]

## Feature Engineering

In [32]:
df['antiguedad'].unique()

array(['      6', '     35', '     34', '     33', '     31', '     21',
       '     16', '     27', '      9', '     22', '     13', '     29',
       '      8', '     11', '     10', '     28', '     24', '      7',
       '     25', '     14', '     26', '     12', '     23', '      1',
       '     18', '      4', '      3', '     17', '     32', '     30',
       '     20', '     15', '     19', '    157', '     36', '      5',
       '     40', '     38', '     37', '     39', '      0', '      2',
       '     47', '     44', '     42', '     46', '     45', '     43',
       '     41', '     57', '     48', '     52', '     49', '     50',
       '     56', '     58', '     51', '     55', '     54', '     53',
       '     59', '     62', '     61', '     60', '     63', '    139',
       '    165', '    118', '    164', '    142', '     94', '    159',
       '    143', '    105', '    151', '    162', '    137', '    150',
       '    128', '    122', '    156', '    119', 

In [28]:
df[df['antiguedad'] == -999999]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
